In [1]:
import copy
import sys
import os
from functools import partial

import geopandas as gpd
from pandas import IndexSlice as idx
import numpy as np
import tensorflow as tf
import gpflow
from tensorflow.python.data.ops.iterator_ops import OwnedIterator as DatasetOwnedIterator

#from metrics import fixed_top_X
#from model_runner import run_adam

code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)

from perturbations import perturbed
from bpr import bpr_variable_k_no_ties

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood,OnOffSVGPBatch
gpflow.config.default_float()

2023-05-22 20:37:19.456239: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 20:37:19.458320: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 20:37:19.508072: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 20:37:19.509143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 20:37:22.239959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

numpy.float64

In [2]:
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

In [3]:
data_path = os.path.join(data_dir, 'clean_annual_tract')
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'
last_train_year = 2018
first_train_year = 2000
test_years = 2
use_auto = False
use_svi = True
seed=360
inducing_points = 200
learning_rate = 0.001
minibatch_size = 100

sigma = 0.05
bpr_samples = 37
noise='normal'

In [4]:
x_idx_cols = [geography_col, 'lat','lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile','year',
              'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)



In [5]:
time_window = 10
first_train_eval_year = 2014
last_train_eval_year = 2018

In [9]:
multiindexed_gdf = data_gdf.set_index(['geoid','year'])
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(['deaths']))

xs =[]
ys = []

for eval_year in range(first_train_eval_year, last_train_eval_year+1):
    
    train_x_df = multiindexed_gdf.loc[idx[:,eval_year-time_window:eval_year-1], 'deaths']
    train_y_df = multiindexed_gdf.loc[idx[:,eval_year], 'deaths']
    
    train_x_vals = train_x_df.values.reshape(train_shape)

    train_y_vals = train_y_df.values
    
    xs.append(train_x_vals)
    ys.append(train_y_vals)
    
    
x_BSTD = np.stack(xs,axis=0)
y_BS = np.stack(ys)

x_BSTD = tf.convert_to_tensor(x_BSTD, dtype=tf.float32)
y_BS = tf.convert_to_tensor(y_BS, dtype=tf.float32)

B, S, T, D = x_BSTD.shape

assert(B==len( range(first_train_eval_year, last_train_eval_year+1)))
assert(S==num_geoids)
assert(T==time_window)
assert(D==1)

# Reshape the training data to flatten the dimensions
x_BSF_flat = tf.reshape(x_BSTD, (B, S, T * D), )

In [12]:
def top_k_idx(input_BD, **kwargs):
    
    _, idx_BD = tf.math.top_k(input_BD, **kwargs)
    input_depth = input_BD.shape[-1]
    one_hot_idx_BKD = tf.one_hot(idx_BD, input_depth)
    #Sum over k dimension so we dont have to worry about sorting
    k_hot_idx_BD = tf.reduce_sum(one_hot_idx_BKD, axis=-2)
    
    
    return k_hot_idx_BD

top_100_idx = partial(top_k_idx, k=100)
perturbed_top_100 = perturbed(top_100_idx,
                         num_samples=bpr_samples,
                         sigma=sigma,
                         noise=noise,
                         batched=True)

class PerturbedBPRModel(tf.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            top_100_indicators = perturbed_top_100(y_pred)
            true_top_100_val, true_top_100_idx = tf.math.top_k(y,k=100)
            
            denominator = tf.reduce_sum(true_top_100_val, axis=-1)
            numerator = tf.reduce_sum(top_100_indicators*y, axis=-1)
            
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [16]:
# Create a TensorFlow model
linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(T, input_shape=(S,T * D,), activation='relu',
                          ),
    tf.keras.layers.Dense(1, input_shape=(S,T ), activation=None,
                          )
])

# Functional api
inputs = tf.keras.Input(shape=(S,T*D))
outputs = tf.squeeze(linear_model(inputs),axis=-1)
model = PerturbedBPRModel(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

# Compile the model
def weird_loss(a,b):
    return -a/b

model.compile(optimizer=optimizer, loss=weird_loss)



In [18]:
# Train the model
model.fit(x_BSF_flat, y_BS, epochs=1000, batch_size=5)  

Epoch 1/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5124
Epoch 2/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5124
Epoch 3/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5113
Epoch 4/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5117
Epoch 5/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5108
Epoch 6/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5098
Epoch 7/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5102
Epoch 8/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5113
Epoch 9/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5109
Epoch 10/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5115
Epoch 11/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5114
Epoch 12/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5115
Epoch 13/1000
1/1 [======

1/1 [==============================] - 0s 45ms/step - loss: -0.5016
Epoch 102/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5013
Epoch 103/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5003
Epoch 104/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5002
Epoch 105/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5000
Epoch 106/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.4989
Epoch 107/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4993
Epoch 108/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4997
Epoch 109/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5001
Epoch 110/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5006
Epoch 111/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4999
Epoch 112/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.4996
Epoch 113/1000
1/1 

1/1 [==============================] - 0s 45ms/step - loss: -0.5035
Epoch 200/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5031
Epoch 201/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5030
Epoch 202/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5032
Epoch 203/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5017
Epoch 204/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5032
Epoch 205/1000
1/1 [==============================] - 0s 42ms/step - loss: -0.5027
Epoch 206/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5027
Epoch 207/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5029
Epoch 208/1000
1/1 [==============================] - 0s 42ms/step - loss: -0.5020
Epoch 209/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5021
Epoch 210/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5014
Epoch 211/1000
1/1 

1/1 [==============================] - 0s 43ms/step - loss: -0.5092
Epoch 298/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5095
Epoch 299/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5096
Epoch 300/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5110
Epoch 301/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5116
Epoch 302/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5116
Epoch 303/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5119
Epoch 304/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5113
Epoch 305/1000
1/1 [==============================] - 0s 42ms/step - loss: -0.5111
Epoch 306/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5121
Epoch 307/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5112
Epoch 308/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5112
Epoch 309/1000
1/1 

1/1 [==============================] - 0s 45ms/step - loss: -0.5121
Epoch 396/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5112
Epoch 397/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5111
Epoch 398/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5104
Epoch 399/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5085
Epoch 400/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5079
Epoch 401/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5081
Epoch 402/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5072
Epoch 403/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5068
Epoch 404/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5063
Epoch 405/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5062
Epoch 406/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5052
Epoch 407/1000
1/1 

...

1/1 [==============================] - 0s 43ms/step - loss: -0.4878
Epoch 592/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4888
Epoch 593/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4885
Epoch 594/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4888
Epoch 595/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4892
Epoch 596/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4890
Epoch 597/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4886
Epoch 598/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4880
Epoch 599/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.4889
Epoch 600/1000
1/1 [==============================] - 0s 42ms/step - loss: -0.4878
Epoch 601/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.4890
Epoch 602/1000
1/1 [==============================] - 0s 42ms/step - loss: -0.4876
Epoch 603/1000
1/1 

1/1 [==============================] - 0s 43ms/step - loss: -0.5036
Epoch 690/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5039
Epoch 691/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5036
Epoch 692/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5035
Epoch 693/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5030
Epoch 694/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5029
Epoch 695/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5035
Epoch 696/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5030
Epoch 697/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5034
Epoch 698/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5031
Epoch 699/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5033
Epoch 700/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5032
Epoch 701/1000
1/1 

1/1 [==============================] - 0s 45ms/step - loss: -0.5016
Epoch 788/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5013
Epoch 789/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5013
Epoch 790/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5010
Epoch 791/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5022
Epoch 792/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5017
Epoch 793/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5014
Epoch 794/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5004
Epoch 795/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4998
Epoch 796/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.5004
Epoch 797/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5006
Epoch 798/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5012
Epoch 799/1000
1/1 

1/1 [==============================] - 0s 44ms/step - loss: -0.5028
Epoch 886/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5030
Epoch 887/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5033
Epoch 888/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5021
Epoch 889/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.5028
Epoch 890/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5033
Epoch 891/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5033
Epoch 892/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5045
Epoch 893/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5044
Epoch 894/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.5043
Epoch 895/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5054
Epoch 896/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.5045
Epoch 897/1000
1/1 

1/1 [==============================] - 0s 45ms/step - loss: -0.4894
Epoch 984/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4891
Epoch 985/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4894
Epoch 986/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4889
Epoch 987/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4891
Epoch 988/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4893
Epoch 989/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4897
Epoch 990/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4889
Epoch 991/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.4889
Epoch 992/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.4886
Epoch 993/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.4900
Epoch 994/1000
1/1 [==============================] - 0s 43ms/step - loss: -0.4907
Epoch 995/1000
1/1 

In [ ]:
model